In [ ]:
# !pip install tf-nightly
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
RANDOM_SEED = 42
columns = ['x-axis', 'y-axis', 'z-axis','activity']
df = pd.read_csv('../input/sisfall/SISFALL_5CLASS.csv', header = 0, names = columns)
df = df.dropna()
df

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
df1=df[['x-axis', 'y-axis', 'z-axis']]
df1=df1*(32.0 / 8192.0)
y = df[['activity']]
df1['activity'] = y.values

scale_columns = ['x-axis', 'y-axis', 'z-axis']
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(df1[scale_columns])
df1.loc[:, scale_columns] = scaler.transform(df1[scale_columns].to_numpy())
values = df1[['x-axis', 'y-axis', 'z-axis']]
labels = df1['activity']
# X['activity'] = y.values
# values=X[['x-axis', 'y-axis', 'z-axis']]
# labels=X['activity']
values

In [ ]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 200
STEP = 40
##############################  CHECK THISSSSSSSSSSSSSS SAIF!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Xdata, Ylabel = create_dataset(
    values, 
    labels, 
    TIME_STEPS, 
    STEP)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc = enc.fit(Ylabel)
Ylabel = enc.transform(Ylabel)

X_train, X_test, y_train, y_test = train_test_split(
        Xdata, Ylabel, test_size=0.2, random_state=RANDOM_SEED,stratify=Ylabel)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
(X_train)

In [ ]:
from tensorflow.keras.models import load_model

# model = load_model('../input/lstmfall/saved_model/1')
# from keras.models import load_model
# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# model.summary()
# def estimate(modelName,inputSignal):
#     model = load_model(modelName)
#     yhat = model.predict(inputSignal)
    
#     indices = []
#     for i in range(yhat.shape[0]):
#         maxIndex = np.where(yhat[i] == np.amax(yhat[i]))[0]
#         indices.append(maxIndex)
        
#         print('\nClass of Object Detected for Signal ' + str(i+1) + ' :')
#         print('Wall = ' + str(round(yhat[i][0]*100,2)) + '%')
#         print('Human = ' + str(round(yhat[i][1]*100,2)) + '%')
#         print('Car = ' + str(round(yhat[i][2]*100,2)) + '%')

#     y_pred = np.vstack(indices)
    
#     return y_pred

In [ ]:
# from sklearn.metrics import accuracy_score
# import matplotlib.pyplot as plt
# y_pred = model.predict(X_test,verbose=1)
# evaluate predictions
# acc = accuracy_score(y_test, yhat)
# print(acc)

# connect predictions with outputs
# for i in range(2):
# #     plt.plot(X_train[i])
# # #     plt.plot((yhat[i]*100).round())
# #     plt.show()
# # # 	print(X_test[i], (yhat[i]*100).round())
#     print(len(X_train[i]))

In [ ]:
pip install --upgrade keras

In [ ]:
from keras.callbacks import ModelCheckpoint
import os
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,LSTM,Dropout,BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras.layers import Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

BATCH_SIZE = 128
model = Sequential()
model.add(BatchNormalization())
model.add(LSTM(256, return_sequences=True, input_shape=([X_train.shape[1], X_train.shape[2]])))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(50))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))  
# opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=220, batch_size=BATCH_SIZE, validation_data=(X_val,y_val),verbose=1, shuffle=True)

In [ ]:
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)
_, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(accuracy)

In [ ]:
import pathlib
# export_dir = '../input/lstmfall/saved_model/1'


# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
#                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)
    

In [ ]:
# # y_pred=model.predict(X_test)
# import pathlib
# export_dir = '../input/lstmfall/saved_model/1'

# mode = "Speed" 

# if mode == 'Storage':
#     optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
# elif mode == 'Speed':
#     optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
# else:
#     optimization = tf.lite.Optimize.DEFAULT

# # EXERCISE: Use the TFLiteConverter SavedModel API to initialize the converter

# converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# # converter.allow_custom_ops = True

# # Set the optimzations
# converter.optimizations = [optimization]
# converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]
# # Invoke the converter to finally generate the TFLite model
# tflite_model = converter.convert()

# tflite_model_file = pathlib.Path('./model.tflite')
# tflite_model_file.write_bytes(tflite_model)


In [ ]:
# print(tf.__version__)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_cm(y_true, y_pred, class_names):
  cm = confusion_matrix(y_true, y_pred)
  fig, ax = plt.subplots(figsize=(18, 16)) 
  ax = sns.heatmap(
      cm, 
      annot=True, 
      fmt="d", 
      cmap=sns.color_palette("muted"),
      ax=ax)

  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  ax.set_xticklabels(class_names)
  ax.set_yticklabels(class_names)
  b, t = plt.ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  plt.ylim(b, t) # update the ylim(bottom, top) values
  #plt.show() # ta-da!
  plt.savefig("confusion_matrix_lstm.png")
plot_cm(enc.inverse_transform(y_test),enc.inverse_transform(y_pred),
        ['D01', 'D02', 'D03', 'D04', 'D05','F01','F02','F03','F04','F05'])

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.figure(dpi=1200)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
plt.savefig('loss_val_loss_lstm.png')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(len(loss))
plt.figure(dpi=1200)
plt.plot(epochs, acc, 'bo', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and validation Accuracy')
plt.legend()
plt.show()
plt.savefig('acc_val_lstm.png')

In [ ]:

# Load TFLite model and allocate tensors.
# interpreter = tf.lite.Interpreter(model_content=tflite_model)
# interpreter.allocate_tensors()
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()
# interpreter.set_tensor(input_details[0]['index'], input_data)
# interpreter.invoke()
# # Get input and output tensors.

# output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
# import os
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense,Flatten,LSTM,Dropout
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint
# import tensorflow as tf

# verbose, epochs, batchSize,nHidden1,nHidden2 = 1, 80, 128,80,30

# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# import keras
# N_TIME_STEPS, N_FEATURES, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
# model = Sequential()
# model.add(LSTM(nHidden1, input_shape=(N_TIME_STEPS,N_FEATURES)))
# model.add(Dropout(0.7))
# model.add(Dense(nHidden2, activation='tanh'))
# model.add(Dense(n_outputs, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# # fit network
# # model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
# # evaluate model
# # _, accuracy = model.evaluate(X_test, y_train, batch_size=batch_size, verbose=1)
# # print(accuracy)


# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,save_best_only=True,
#                                                  verbose=1)

# # callbacks= [ModelCheckpoint('model.h5', save_weights_only=False, save_best_only=True, verbose=1)]
# history = model.fit(X_train,y_train,shuffle=False,epochs=epochs,verbose=verbose, validation_data=(X_val,y_val), callbacks=cp_callback)

In [ ]:
# _, accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(accuracy)

In [ ]:
# from tensorflow.keras.models import load_model
# model=load_model('./model.h5')
# model.summary()
# model.save("model")
# model=load_model('model')
# loss, acc = model.evaluate(X_test, y_test, verbose=1)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
# pd.DataFrame(history.history).plot(figsize=(25, 25))
# plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
# plt.show()

In [ ]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import confusion_matrix
# y_test_arg=np.argmax(y_test,axis=1)
# Y_pred = np.argmax(model.predict(X_test),axis=1)
# # predictions = model.predict(X_test)
# # cm = confusion_matrix(y_test_arg, Y_pred)
# # plot_confusion_matrix(cm, X_test,np.unique(Y_pred))
# print(confusion_matrix(y_test_arg, Y_pred))
# plot_confusion_matrix(cm, np.argmax(X_test,axis=1),np.unique(Y_pred))

In [ ]:
from sklearn.metrics import cohen_kappa_score, f1_score, confusion_matrix
cm = confusion_matrix(enc.inverse_transform(y_test),enc.inverse_transform(y_pred))

def evaluate_metrics(confusion_matrix, y_test, y_pred, print_result=False, f1_avg='macro'):
    # https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
    
    TP = np.diag(confusion_matrix)
    FP = confusion_matrix.sum(axis=0) - TP
    FN = confusion_matrix.sum(axis=1) - TP    
    TN = confusion_matrix.sum() - (FP + FN + TP)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP / (TP + FN)
    # Specificity or true negative rate
    TNR = TN / (TN + FP)
    # Precision or positive predictive value
    PPV = TP / (TP + FP)
    # Negative predictive value
    NPV = TN / (TN + FN)
    # Fall out or false positive rate
    FPR = FP / (FP + TN)
    # False negative rate
    FNR = FN / (TP + FN)
    # False discovery rate
    FDR = FP / (TP + FP)

    # Overall accuracy
    ACC = (TP + TN) / (TP + FP + FN + TN)
    # ACC_micro = (sum(TP) + sum(TN)) / (sum(TP) + sum(FP) + sum(FN) + sum(TN))
    ACC_macro = np.mean(
        ACC)  # to get a sense of effectiveness of our method on the small classes we computed this average (macro-average)

    f1 = f1_score(y_test, y_pred, average=f1_avg)
    kappa = cohen_kappa_score(y_test, y_pred)
    
    if (print_result):
        print("\n")
        print("\n")
        print("============ METRICS ============")
        print(confusion_matrix)
        print("Accuracy (macro) : ", ACC_macro)        
        print("F1 score         : ", f1)
        print("Cohen Kappa score: ", kappa)
        print("======= Per class metrics =======")
        print("Accuracy         : ", ACC)
        print("Sensitivity (TPR): ", TPR)
        print("Specificity (TNR): ", TNR)
        print("Precision (+P)   : ", PPV)
    
    return ACC_macro, ACC, TPR, TNR, PPV, f1, kappa

# acc=evaluate_metrics(cm, enc.inverse_transform(y_pred),enc.inverse_transform(y_test),True)